In [1]:
import pandas as pd
import numpy as np
import collections
import time
import math

In [2]:
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=','
                           ,usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties',
                                   'Satisfactory Rating', 'Shower','Washing Machine', 'Fridge','3 Star',
                                    'Air Conditioning','Good Rating','Car Park' ]
                           )
item_metadata['item_id'] = item_metadata['item_id'].astype(int)
item_metadata.shape

(813442, 13)

In [3]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 6000000)
#                    nrows=100000
                   )
train = train[train["action_type"] == "clickout item"]
print(train.shape)
train[:3]

(660653, 13)


,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
15,6000014,de86377f72f0f,1541248401,V2M2533G2TVI,7,clickout item,4936470,AU,"Melbourne, Australia",desktop,Serviced Apartment,109094|55074|101988|8472964|109095|848971|2291...,139|130|91|266|231|122|125|108|161|120|104|106...
34,6000033,0d993d2278335,1541248402,388RD30GUJBJ,3,clickout item,47173,UK,"Eastbourne, United Kingdom",mobile,NaN,47802|1062872|47173|1556531|46676|914675|32280...,148|63|76|71|50|44|63|59|80|71|82|50|62|57|64|...
37,6000036,d02ea044226bc,1541248402,F3W2KEFHJ2QL,16,clickout item,2326194,AR,"Maceió, Brazil",desktop,NaN,2326194|109324|2179366|1321131|111530|478651|2...,226|207|143|185|137|171|135|121|205|83|176|149...


In [4]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [5]:
train = explode_impressions_and_prices(train)
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126


In [6]:
train['position'] = train.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3


In [7]:
# # DODANIE CITY I COUNTRY
# city, country = (train['city']
#                     .values[0]
#                     .replace(" ", "")
#                     .split(','))
# print(city)
# print(country)

# cities = []
# countries = []
# l = []
# for index in range(len(train['city'].values)):
#     l = (train['city']
#                     .values[index]
#                     .replace(" ", "")
#                     .split(','))
#     cities.append(l[0])
#     countries.append(l[1])
# #     if len(l) > 2 :
# #         print(l[0])
# #         print(l[1])
# #         print(l[2])
# #         print(len(l))

# train = train.drop(['city'], axis=1)
# train['city'] = cities
# train['country'] = countries
# train[:3]

In [8]:
def mean_price(df):
    return df['prices'].mean()
df_mean_price = train.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
train = train.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
train['mean_price_difference'] = abs(train['prices'] - train['mean_price'])
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,mean_price,mean_price_difference
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1,82.6,13.4
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2,82.6,1.6
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3,82.6,4.4


In [9]:
train = train.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,...,clicks_views_ratio,Shower,Satisfactory Rating,Fridge,Air Conditioning,Good Rating,Washing Machine,3 Star,Car Park,sum_of_properties
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.052632,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,89.0
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.075658,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,72.0
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.111732,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,75.0


In [10]:
train.columns

Index(['index', 'session_id', 'timestamp', 'user_id', 'step', 'action_type',
       'reference', 'platform', 'city', 'device', 'current_filters', 'prices',
       'number_of_impressions', 'impressions', 'position', 'mean_price',
       'mean_price_difference', 'item_id', 'n_clicks', 'views',
       'clicks_views_ratio', 'Shower', 'Satisfactory Rating', 'Fridge',
       'Air Conditioning', 'Good Rating', 'Washing Machine', '3 Star',
       'Car Park', 'sum_of_properties'],
      dtype='object')

In [11]:
train = train.rename({'index':'item_id'})
train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position','sum_of_properties','Satisfactory Rating', 
               'Shower','Washing Machine', 'Fridge','3 Star',
                'Air Conditioning','Good Rating','Car Park',
              'mean_price','mean_price_difference']]
# train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','sum_of_properties','mean_price','mean_price_difference','position']]
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,sum_of_properties,Satisfactory Rating,Shower,Washing Machine,Fridge,3 Star,Air Conditioning,Good Rating,Car Park,mean_price,mean_price_difference
0,5479306,5479306,2.0,38,0.052632,1,89.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,13.4
1,97126,5479306,23.0,304,0.075658,2,72.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,1.6
2,2195126,5479306,20.0,179,0.111732,3,75.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,4.4


In [12]:
train['reference'] = train['reference'].astype(int)
train['clicked'] = np.where(train['item_id'] == train['reference'], 1, 0)
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,sum_of_properties,Satisfactory Rating,Shower,Washing Machine,Fridge,3 Star,Air Conditioning,Good Rating,Car Park,mean_price,mean_price_difference,clicked
0,5479306,5479306,2.0,38,0.052632,1,89.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,13.4,1
1,97126,5479306,23.0,304,0.075658,2,72.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,1.6,0
2,2195126,5479306,20.0,179,0.111732,3,75.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,82.6,4.4,0


In [ ]:
# train.drop_duplicates('item_id', inplace=True)
# collections.Counter(train['item_id'].values)

In [ ]:
# collections.Counter(train['clicked'].values)

In [ ]:
# import plotly.express as px
# fig = px.scatter(train[:10000],x='n_clicks',y='views', color='clicked')
# fig.show()

In [13]:
train = train.drop(columns=['reference','mean_price'])
train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,sum_of_properties,Satisfactory Rating,Shower,Washing Machine,Fridge,3 Star,Air Conditioning,Good Rating,Car Park,mean_price_difference,clicked
0,5479306,2.0,38,0.052632,1,89.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,13.4,1
1,97126,23.0,304,0.075658,2,72.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.6,0
2,2195126,20.0,179,0.111732,3,75.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,4.4,0


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[['item_id', 'Satisfactory Rating', 
                                                           'Shower','Washing Machine', 'Fridge','3 Star',
                                                            'Air Conditioning','Good Rating','Car Park','clicks_views_ratio',
                                                           'position','mean_price_difference']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

,item_id,Satisfactory Rating,Shower,Washing Machine,Fridge,3 Star,Air Conditioning,Good Rating,Car Park,clicks_views_ratio,position,mean_price_difference
4730222,2855622,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.048128,21,26.541667
8297644,917339,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.012788,13,59.120000
7754098,2630756,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.027778,19,40.720000


In [35]:
X_train_id_columns = X_train.reset_index().drop(columns=['index'], axis=1)[['item_id','Satisfactory Rating', 
                                                           'Shower','Washing Machine', 'Fridge','3 Star',
                                                            'Air Conditioning','Good Rating','Car Park']]
X_train_rest_columns = X_train.reset_index().drop(columns=['index'], axis=1).iloc[:,9:]

In [36]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train_rest_columns)
X_train_rest_columns = sc.transform(X_train_rest_columns)
# X_test = sc.transform(X_test)
X_train_rest_columns[:3]

array([[ 0.08172692,  1.18799864, -0.16858364],
       [-0.59542798,  0.06986454,  0.07943365],
       [-0.30820639,  0.90846511, -0.06064467]])

In [37]:
X_train = np.hstack((X_train_id_columns, X_train_rest_columns))
X_train

array([[ 2.85562200e+06,  1.00000000e+00,  0.00000000e+00, ...,
         8.17269215e-02,  1.18799864e+00, -1.68583645e-01],
       [ 9.17339000e+05,  1.00000000e+00,  1.00000000e+00, ...,
        -5.95427981e-01,  6.98645416e-02,  7.94336464e-02],
       [ 2.63075600e+06,  1.00000000e+00,  1.00000000e+00, ...,
        -3.08206393e-01,  9.08465115e-01, -6.06446723e-02],
       ...,
       [ 4.01681000e+06,  1.00000000e+00,  0.00000000e+00, ...,
        -9.13066556e-02, -1.04826956e+00, -2.33610944e-01],
       [ 2.65063800e+06,  1.00000000e+00,  1.00000000e+00, ...,
        -4.30594222e-01, -6.99022206e-02,  1.59217384e-01],
       [ 2.10850000e+04,  1.00000000e+00,  1.00000000e+00, ...,
         1.36878808e-01, -1.60733660e+00,  4.04963478e-01]])

In [38]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=12))
# model.add(Dense(70, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model.fit(X_train, y_train.values, epochs=30, batch_size=50)

Epoch 1/30
12144580/12144580 [==============================] - 244s 20us/step - loss: nan - accuracy: 1.3487e-04 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 2/30
12144580/12144580 [==============================] - 230s 19us/step - loss: nan - accuracy: 0.0000e+00 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 3/30
12144580/12144580 [==============================] - 222s 18us/step - loss: nan - accuracy: 0.0000e+00 - f1_m: nan - precision_m: nan - recall_m: nans - loss: nan - accuracy: 0.0000e+00 - f1_m: nan - precision_m: nan - rec
Epoch 4/30
12144580/12144580 [==============================] - 221s 18us/step - loss: nan - accuracy: 0.0000e+00 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 5/30
12144580/12144580 [==============================] - 222s 18us/step - loss: nan - accuracy: 0.0000e+00 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 6/30
12144580/12144580 [==============================] - 222s 18us/step - loss: nan - accuracy: 0.0000e+00 - f1_m: 

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


# NAUKA

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
nans = lambda df: df[df.isnull().any(axis=1)]
print(len(train))
train = train.dropna()
print(len(train))

In [ ]:
# 'n_clicks','views','clicks_views_ratio', 'mean_price','mean_price_difference','position'
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,0:-1], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
# sc = StandardScaler()
# sc.fit(X_train)
# X_train = sc.transform(X_train)
# # X_train = normalize(X_train, norm='l2')
# X_test = sc.transform(X_test)
# # X_test = normalize(X_test, norm='l2')

In [ ]:
# logreg = DecisionTreeClassifier(class_weight={0.0:0.8, 1.0:0.2})
logreg = LogisticRegression(n_jobs=-1, class_weight={0.0:0.1, 1.0:0.9}) #class_weight={0.0:0.2, 1.0:0.8}
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# y_pred_prob = logreg.predict_proba(X_test)

In [ ]:
print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

# ZBIÓR TESTOWY

In [ ]:
test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\test.csv',
                       sep=',',
#                     skiprows=range(1, 11000000)
#                    nrows=10000
                   )
# test = test[test["action_type"] == "clickout item" & test['reference'].isnull()]
mask = test["reference"].isnull() & (test["action_type"] == "clickout item")
test = test[mask]
# print(test.shape)
test[:3]

In [ ]:
test = explode_impressions_and_prices(test)
test[:3]

In [ ]:
test['position'] = test.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
test[:3]

In [ ]:
index_df = test[["user_id","session_id","timestamp","step","impressions"]]
index_df[:10]

In [ ]:
test = test.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
test[:3]

In [ ]:
test.columns

In [ ]:
train.columns

In [ ]:
test = test.rename({'index':'item_id'})
test = test[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position','Satisfactory Rating', 
               'Shower','Washing Machine', 'Fridge','3 Star',
                'Air Conditioning','Good Rating','Car Park']]
test = test.drop(columns=['reference'], axis=1)
test = test.dropna()
test[:3]

In [ ]:
X_test = test
# X_test = sc.transform(X_test)

In [ ]:
y_pred = logreg.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
result = pd.concat([index_df, pd.DataFrame(y_pred)], axis=1, sort=False)
# result = result.drop([0], axis=1)
result[:10]

In [ ]:
result = result.assign(impressions=lambda x: x["impressions"].apply(str))
result = result.sort_values(["user_id", "session_id", "timestamp", "step", 1],
                     ascending=[True, True, True, True, False])
result[:10]

In [ ]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [ ]:
result = group_concat(result, ["user_id","session_id","timestamp","step"], "impressions")
result = result.rename(columns={"impressions":"item_recommendations"})
result[:10]

In [ ]:
result.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_ML.csv", index=False,)